In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'

gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


In [ ]:
!nvidia-smi -L

# Natural language processing

## Download dataset

In [ ]:
train_df = pd.read_csv("../input/nlp-getting-started/train.csv")
test_df = pd.read_csv("../input/nlp-getting-started/test.csv")
train_df.head()

In [ ]:
train_df_shuffled = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
train_df_shuffled.head()

## Splitting test and validation

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = (
    train_test_split(train_df_shuffled["text"].to_numpy(),
                     train_df_shuffled["target"].to_numpy(),
                     test_size=0.1,
                     random_state=42)
)

len(X_train), len(X_test), len(y_train), len(y_test)

## Preprocessing using `tf.data`

In [ ]:
from tensorflow.keras import models, layers, callbacks
from tensorflow.keras.optimizers import Adam

BATCH_SIZE = 32

def prepare(ds, shuffle=False):
    
    # Applying normalization before `ds.cache()` to re-use it.
    # Note: Random transformations (e.g. images augmentations) should be applied
    # after both `ds.cache()` (to avoid caching randomness) and `ds.batch()` (for
    # vectorization [1]).

    if shuffle:
        ds = ds.shuffle(buffer_size=1000)

    # Batch after shuffling to get unique batches at each epoch.
    ds = ds.batch(BATCH_SIZE)
    ds = ds.cache() 

    return ds.prefetch(buffer_size=tf.data.AUTOTUNE)


In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_data = prepare(train_data, shuffle=True)

test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_data = prepare(test_data)

# Baseline Model

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tdidf", TfidfVectorizer()),
    ("model", MultinomialNB())  
])

model_0.fit(X_train,y_train)

In [ ]:
baseline_score = model_0.score(X_test,y_test)
print(f"Baseline score: {baseline_score*100:.2f}%")

## Evaluating the model

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.
  Args:
      y_true: true labels in the form of a 1D array
      y_pred: predicted labels in the form of a 1D array
  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [ ]:
y_pred = model_0.predict(X_test)
baseline_results = calculate_results(y_true=y_test, y_pred=y_pred)
baseline_results


# Transfer Learning

## Create model

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy(policy='mixed_float16')
mixed_precision.global_policy()

In [ ]:
import tensorflow_hub as hub

USE5_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5"

encoder_layer = hub.KerasLayer(
    USE5_url,
    output_shape=[],
    dtype=tf.string,
    trainable=False,
    name="USE"
)


In [ ]:
model = models.Sequential([
    layers.Input(shape=[], dtype=tf.string),
    encoder_layer,
    layers.Dense(128, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(64, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(1),
    layers.Activation("sigmoid", dtype=tf.float32)
], name="model_USE")


model.compile(optimizer=Adam(learning_rate=2.5e-4),
                loss="binary_crossentropy",
                metrics=["accuracy"])
model.summary()

In [ ]:
for layer in model.layers:
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

In [ ]:
stop_callback = callbacks.EarlyStopping(monitor="val_loss",
                                        patience=3)

lr_callback = callbacks.ReduceLROnPlateau(monitor="val_loss",
                                          factor=0.5,
                                          patience=2,
                                          verbose=1,
                                          min_lr=1e-7)

checkpoint_callback = callbacks.ModelCheckpoint('saved_models/disaster',
                                                monitor='val_accuracy',
                                                save_best_only=True)

history_model = model.fit(train_data,
                          epochs=100,
                          validation_data=(test_data),
                          callbacks=[stop_callback, lr_callback, checkpoint_callback])


## Evaluating the model

In [ ]:
model_preds_prob = model.predict(X_test)
model_preds = model_preds_prob.round().squeeze()
model_preds[:10]

In [ ]:
model_results = calculate_results(y_test,model_preds)
model_results


In [ ]:
def compare_baseline_to_new_results(baseline_results, new_model_results):
  for key, value in baseline_results.items():
    print(f"Baseline {key}: {value:.2f}, New {key}: {new_model_results[key]:.2f}, Difference: {new_model_results[key]-value:.2f}")

compare_baseline_to_new_results(baseline_results,model_results)

# Comparing two Models

In [ ]:
all_results = pd.DataFrame({
    "base_model": baseline_results,
    "hub-encoder": model_results
})
all_results = all_results.transpose()
all_results["accuracy"] = all_results["accuracy"]/100
all_results.plot(kind="bar", figsize=(9,9)).legend(bbox_to_anchor=(1., 1))


In [ ]:
all_results.sort_values(by="f1", ascending=False)["f1"].plot(kind="bar", figsize=(5,5)).set_title("f1-score")

# Predictions


In [ ]:
test_data = test_df.text.values
test_pred = model.predict(test_data)

In [ ]:
submission_df = pd.DataFrame({
    "id": test_df.id,
    "target": test_pred.round().squeeze().astype(np.int64)
})

In [ ]:
submission_df.to_csv("submission.csv", index=False)

# Load model

In [ ]:
loaded_model = models.clone_model(model)
loaded_model.load_weights("saved_models/disaster")
loaded_model.compile(optimizer=Adam(learning_rate=2.5e-4),
                     loss="binary_crossentropy",
                     metrics=["accuracy"])


In [ ]:
loaded_model_preds_prob = model.predict(X_test)
loaded_model_preds = model_preds_prob.round().squeeze()
loaded_model_preds[:10]

In [ ]:
loaded_model_results = calculate_results(y_test,loaded_model_preds)
loaded_model_results